In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import time
import math

from torch.utils.data import DataLoader

import torch
import torch.nn as nn

%matplotlib inline

torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

gpu = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu

import sys
# Get the previous folder and append it to sys.path
PATH = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PATH)

from config import DATAFOLDER

In [2]:
src_domain = 'FSeg'
tgt_domain = 'Thebe'

da_method = 'FDA'
Lb = 0.1
fda_imgs = 1024

# data
red = 0 # reduction factor
n_imgs = 16000  # {16k, 32k} max: 51200
bs = 16 # batch size
type_aug = "type1_fda" # type0, type1
nw = 1 # num_workers

# model
net = "dexined" # unet2d / dexined / at-unet
f = 16 # filters on first layer
n_blocks = 3  # blocks on encode / decode -> unet based
bn = False # batch normalization
dropout = False

# training
optimizer = 'Adam'
lr = 1e-4
lr_adjust = False
epochs = 10 # 100
# loss = 'BCE' # unet based
loss = 'bdcn' # dexined

# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load Data

In [4]:
from dataloaders import ThebeGenerator, FaultSeg2DGenerator

In [5]:
dataset = 'Thebe'
DATAPATH = os.path.join(DATAFOLDER, dataset)

ps = '256to128' # patch size
step = 3 # step size

DATAPATH_TH_SRC = os.path.join(DATAPATH, 'Patches2D', 'src_ps{}_step{}'.format(ps, step) )
DATAPATH_TH_TGT = os.path.join(DATAPATH, 'Patches2D', 'tgt_ps{}_step{}'.format(ps, step) )

train_gen_TH = ThebeGenerator(DATAPATH_TH_TGT, "train", red=red, n_imgs=n_imgs) #, aug=type_aug)
val_gen_TH = ThebeGenerator(DATAPATH_TH_SRC, "val", red=red, n_imgs=n_imgs)
len(train_gen_TH.list_IDs) # max. 49770

16000

In [6]:
# extract references images from target domain
ref_imgs=[]
for idx, (x,_) in enumerate(train_gen_TH):
    ref_imgs.append(x[0])
    if idx == fda_imgs:
        break

In [7]:
dataset = 'FaultSeg3D'
DATAPATH_FS = os.path.join(DATAFOLDER, dataset)

train_gen_FS = FaultSeg2DGenerator(DATAPATH_FS, "train", red=red, n_imgs=n_imgs, aug=type_aug,
                                  ref_imgs=ref_imgs, Lb=Lb)
val_gen_FS = FaultSeg2DGenerator(DATAPATH_FS, "val", red=red, n_imgs=n_imgs)
len(train_gen_FS.list_IDs) # 51200

16000

In [8]:
# shuffle = False
train_tgt = DataLoader(dataset=train_gen_TH, batch_size=bs, shuffle=False, drop_last=True, num_workers=nw)
val_tgt = DataLoader(dataset=val_gen_TH, batch_size=bs, shuffle=False, drop_last=True, num_workers=nw)

train_src = DataLoader(dataset=train_gen_FS, batch_size=bs, shuffle=True, drop_last=True, num_workers=nw)
val_src = DataLoader(dataset=val_gen_FS, batch_size=bs, shuffle=False, drop_last=True, num_workers=nw)

## Modeling

In [9]:
from models.unet import UNet2D
from models.dexined import DexiNed
from models.at_unet import AtrousUNet

from loss import bdcn_loss2

In [10]:
if net == "unet2d":
    model = UNet2D(f1=f).to(device)
elif net == "at-unet": 
    model = AtrousUNet(f1=f, depth=4).to(device)
elif net == "dexined": 
    model = DexiNed(f1=f).to(device)
else:
    print("Error")

In [11]:
opt = torch.optim.Adam(model.parameters(), lr=lr)

criterion = nn.BCELoss()

if net == "dexined":
    criterion = bdcn_loss2

## Training

In [12]:
from metrics import *

In [13]:
# saved_name = wandb.run.id
# saved_path = os.path.join(PATH, 'res', '{}.pt'.format(saved_name))
# wandb.config.update({"saved_path": saved_name})

saved_path = "delete.pt"

stopPatience = 10
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', factor=0.1, patience=stopPatience//2, verbose=True)
saved_path

'delete.pt'

In [14]:
def process(load_src, load_tgt, model, phase='train', opt=None, epoch=0):

    task_loss_ = 0.0
    
    iou_ = 0.0
    iou_tgt_ = 0.
    iou_tgt = 0. # NO DELETE
    
    if phase=='train':
        model.train()  
    else:
        model.eval()
        
    epoch += 1
    
    if phase=='train':
        model.train() 
        if lr_adjust:
            for param_group in opt.param_groups:
                param_group['lr'] = lr / math.pow((1 + 10 * (epoch - 1) / epochs), 0.75)
    else:
        model.eval()
    
    for step_, ((x_src, y_src), (x_tgt, y_tgt)) in enumerate(zip(load_src, load_tgt)):
        
        # make images variable
        x_src = x_src.float().to(device=device)
        y_src = y_src.to(device=device).unsqueeze(1)

        # source domain
        src_class_output = model(x_src)

        if net == "dexined":

            src_loss_class = criterion(src_class_output[-1], y_src, 0.5) + criterion(src_class_output[2], y_src, 0.5) # bdcn_loss
            src_loss_class = src_loss_class / 5
            
            src_class_output = (src_class_output[-1] + src_class_output[2] + src_class_output[3])/3.
        else:
            src_loss_class = criterion(src_class_output, y_src)
        
        if phase=='train':
            # calculate the gradients
            opt.zero_grad()
            src_loss_class.backward()

            # update weigths
            opt.step()
        
        ## compute loss
        task_loss_ += src_loss_class.item()
        
        ## compute metrics
        pred_ = (src_class_output > 0.5).float() 
        iou_ += compute_iou(pred_.squeeze(1), y_src.squeeze(1))
        
        if phase=='val':

            # make images variable
            x_tgt = x_tgt.float().to(device=device)
            y_tgt = y_tgt.to(device=device).unsqueeze(1)

            # target domain
            tgt_class_output = model(x_tgt)

            if net == "dexined":
                tgt_class_output = (tgt_class_output[-1] + tgt_class_output[2] + tgt_class_output[3])/3.

            pred_2 = (tgt_class_output > 0.5).float()
            iou_tgt_ += compute_iou(pred_2.squeeze(1), y_tgt.squeeze(1))        
        
    task_loss = task_loss_/(step_+1)
    iou = iou_/(step_+1)
    
    if phase=='val':
        iou_tgt = iou_tgt_/(step_+1)
        scheduler.step(task_loss)
        
    return task_loss, iou, iou_tgt

In [15]:
min_valid_loss = np.inf
max_valid_vl = -np.inf
n_patience = 0
t_start = time.time()
t_start_total = time.time()

for ep in range(epochs):
    
    task_loss_tr, iou_src_tr, iou_tgt_tr = process(train_src, train_tgt, model, phase='train', opt=opt, epoch=ep)

    with torch.no_grad():
        task_loss_vl, iou_src_vl, iou_tgt_vl = process(val_src, val_tgt, model, phase='val', epoch=ep)
        
    t_ = time.time()-t_start

    for param_group in opt.param_groups:
        lr_step = param_group['lr']
    
    # wandb.log({
    #     "train/epoch/time": t_,
    #     "train/epoch/step": ep+1,
    #     "lr/step": lr_step,
        
    #     "train/epoch/task_loss": task_loss_tr,
    #     "train/epoch/iou/src": iou_src_tr,
        
    #     "val/epoch/task_loss": task_loss_vl,
    #     "val/epoch/iou/src": iou_src_vl,
    #     "val/epoch/iou/tgt": iou_tgt_vl

    # })
    

    print('Epoch {}:   {:.1f}s - loss: {:.4f} - iou_src: {:.4f}'.format(ep+1, t_, task_loss_tr, iou_src_tr))
    print('                   val_loss: {:.4f} - val_iou_src: {:.4f} - val_iou_tgt: {:.4f}'.format(task_loss_vl, iou_src_vl, iou_tgt_vl))
#     print('')

    loss_vl = task_loss_vl
    
    if min_valid_loss > loss_vl:
        print(f'val_loss improved from {min_valid_loss:.6f} to {loss_vl:.6f} , saving model\n')
        min_valid_loss = loss_vl
        torch.save(model.state_dict(), saved_path)
        n_patience=0
    else:
        print(f'val_loss did not improve from {min_valid_loss:.6f}\n')
        n_patience+=1
        if n_patience==stopPatience:
            print('Epoch {}:   early stopping'.format(ep+1))
            break
            
    t_start = time.time()
    
t_total = time.time() - t_start_total

Epoch 1:   124.1s - loss: 0.2509 - iou_src: 0.1863
                   val_loss: 0.2445 - val_iou_src: 0.2064 - val_iou_tgt: 0.1088
val_loss improved from inf to 0.244549 , saving model

Epoch 2:   123.9s - loss: 0.2128 - iou_src: 0.3078
                   val_loss: 0.2223 - val_iou_src: 0.3196 - val_iou_tgt: 0.1910
val_loss improved from 0.244549 to 0.222341 , saving model

Epoch 3:   129.0s - loss: 0.1975 - iou_src: 0.3513
                   val_loss: 0.2133 - val_iou_src: 0.3324 - val_iou_tgt: 0.1961
val_loss improved from 0.222341 to 0.213266 , saving model

Epoch 4:   129.7s - loss: 0.1879 - iou_src: 0.3768
                   val_loss: 0.2027 - val_iou_src: 0.3658 - val_iou_tgt: 0.2058
val_loss improved from 0.213266 to 0.202670 , saving model

Epoch 5:   131.1s - loss: 0.1803 - iou_src: 0.3944
                   val_loss: 0.1995 - val_iou_src: 0.3811 - val_iou_tgt: 0.2164
val_loss improved from 0.202670 to 0.199486 , saving model

Epoch 6:   132.2s - loss: 0.1748 - iou_src: 0.4077

In [16]:
t_total/60, ep

(21.77193213303884, 9)

## Validation

In [17]:
model.load_state_dict(torch.load(saved_path))
model.eval()
print()

In [18]:
for idx, (x_tgt, y_tgt) in enumerate(val_tgt):
    print(x_tgt.shape, y_tgt.shape)
    break

torch.Size([16, 1, 128, 128]) torch.Size([16, 128, 128])


In [24]:
y_tgt_prob = model(x_tgt.float().to(device)) #.cpu().detach().numpy()

if net == "dexined":
    y_tgt_prob = (y_tgt_prob[-1] + y_tgt_prob[2] + y_tgt_prob[3])/3.
    y_tgt_prob = torch.nn.Sigmoid()(y_tgt_prob)

y_tgt_prob = y_tgt_prob.cpu().detach().numpy()
y_tgt_pred = np.where(y_tgt_prob[:,0] >= 0.5, 1, 0)
y_tgt_prob.shape, y_tgt_pred.shape

((16, 1, 128, 128), (16, 128, 128))

In [25]:
fig, axes = plt.subplots(4, 8, figsize=[16,6])

for i in range(8):
    axes[0, i].imshow(x_tgt[i,0], cmap="gray")
    axes[1, i].imshow(y_tgt[i], cmap="jet", interpolation="nearest")
    axes[2, i].imshow(y_tgt_prob[i,0])
    axes[3, i].imshow(y_tgt_pred[i], cmap="jet", interpolation="nearest")
    
# wandb.log({"valid/imgs_tgt": fig})

### Metrics

In [26]:
import pandas as pd
from tqdm import tqdm

In [28]:
val_tgt = DataLoader(dataset=val_gen_TH, batch_size=128, shuffle=False, drop_last=True,
                     num_workers=2)

y_true_list = []
y_prob_list = []
for x_tgt, y_tgt in tqdm(val_tgt):
    y_tgt = y_tgt.cpu().detach().numpy()

    y_prob = model(x_tgt.float().to(device))

    if net == "dexined":
        y_prob = (y_prob[-1] + y_prob[2] + y_prob[3])/3.
        y_prob = torch.nn.Sigmoid()(y_prob)

    y_prob = y_prob.cpu().detach().numpy()
    
    y_true_list.append(y_tgt)
    y_prob_list.append(y_prob[:,0])
    
y_true_total = np.vstack(y_true_list) #.ravel()
y_prob_total = np.vstack(y_prob_list) #.ravel()
y_pred_total = np.where(y_prob_total >= 0.5, 1, 0)

y_true_total.shape

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:06<00:00,  5.29it/s]


(4352, 128, 128)

In [29]:
th_range = np.arange(0, 1, 0.02)
metrics_list = []

y_true = torch.tensor(y_true_total, dtype=torch.float32).to(device)
y_prob = torch.tensor(y_prob_total, dtype=torch.float32).to(device)

for th in th_range:
    pr, re, f1 = binary_classification_metrics(y_prob, y_true, th)
    metrics_list.append([pr, re, f1])

In [30]:
df = pd.DataFrame(np.array(metrics_list)).reset_index(drop=True)
df.columns = ['precision', 'recall', 'f1_score']
df.index = th_range

In [31]:
f1_ = df.loc[0.5]['f1_score']
# wandb.log({"valid/f1_tgt": f1_})

In [32]:
iou = compute_iou(y_true.contiguous(), (y_prob > 0.5).float())
# wandb.log({"valid/iou_tgt": iou})

In [33]:
print("Original:  IoU: {:.4f}   F1-score: {:.4f}".format(iou, f1_))

Original:  IoU: 0.2566   F1-score: 0.4085


### New Metrics

In [34]:
getNewMetrics(y_true_total, y_pred_total, sigma=1) #, wandb=wandb)

248it [00:00, 616.05it/s]C:\Users\Usuario\Documents\Mestrado\Tesis\src\UDAforFaultDetection\metrics.py:94: RuntimeWarning: invalid value encountered in scalar divide
  return np.sum(d)/np.sum(n)
C:\Users\Usuario\Documents\Mestrado\Tesis\src\UDAforFaultDetection\metrics.py:157: RuntimeWarning: divide by zero encountered in scalar divide
  iou = 1/(1/recall+1/precision-1)
C:\Users\Usuario\Documents\Mestrado\Tesis\src\UDAforFaultDetection\metrics.py:158: RuntimeWarning: divide by zero encountered in scalar divide
  f1 = 2/(1/recall+1/precision)
4352it [00:06, 661.55it/s]

With Sigma=1:  IoU: 0.3250   F1-score: 0.4617


In [35]:
getNewMetrics(y_true_total, y_pred_total, sigma=4) #, wandb=wandb)

4352it [00:05, 771.06it/s]

With Sigma=4:  IoU: 0.5148   F1-score: 0.6462


In [36]:
getNewMetrics(y_true_total, y_pred_total, sigma=7) #, wandb=wandb)

4352it [00:05, 772.35it/s]

With Sigma=7:  IoU: 0.5989   F1-score: 0.7192
